## Extract FLASH Data in yt
Mirielle Wong

6/21/24

Use yt's covering_grid class to extract data from FLASH.

In [62]:
# https://yt-project.org/

import yt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

In [36]:
plot_path = "./OSIRIS_transfer/MAGON/MagShockZ_hdf5_plt_cnt_0004"

In [6]:
ds = yt.load(plot_path)

yt : [INFO     ] 2024-06-21 12:40:25,600 Parameters: current_time              = 1.5001873777002105e-09
yt : [INFO     ] 2024-06-21 12:40:25,600 Parameters: domain_dimensions         = [288 272 288]
yt : [INFO     ] 2024-06-21 12:40:25,601 Parameters: domain_left_edge          = [-0.6   -0.075 -0.6  ]
yt : [INFO     ] 2024-06-21 12:40:25,601 Parameters: domain_right_edge         = [0.6 1.  0.6]
yt : [INFO     ] 2024-06-21 12:40:25,601 Parameters: cosmological_simulation   = 0


In [7]:
list(ds.fields.flash)

[On-Disk Field ('flash', 'cham'): (units: '', display_name: 'Chamber Material Fraction'),
 On-Disk Field ('flash', 'dens'): (units: 'code_mass/code_length**3'),
 On-Disk Field ('flash', 'depo'): (units: 'code_length**2/code_time**2'),
 On-Disk Field ('flash', 'magp'): (units: 'code_mass/(code_length*code_time**2)'),
 On-Disk Field ('flash', 'magx'): (units: 'code_magnetic', display_name: 'B_x'),
 On-Disk Field ('flash', 'magy'): (units: 'code_magnetic', display_name: 'B_y'),
 On-Disk Field ('flash', 'sumy'): (units: ''),
 On-Disk Field ('flash', 'targ'): (units: '', display_name: 'Target Material Fraction'),
 On-Disk Field ('flash', 'tele'): (units: 'code_temperature'),
 On-Disk Field ('flash', 'tion'): (units: 'code_temperature'),
 On-Disk Field ('flash', 'trad'): (units: 'code_temperature'),
 On-Disk Field ('flash', 'ye'): (units: '', display_name: 'Y_e'),
 Alias Field for ('index', 'cell_volume') ('flash', 'volume'): (units: 'cm**3'),
 Alias Field for ('index', 'z') ('flash', 'z'): 

In [37]:
print(ds.domain_dimensions)
domain_left_edge = [-0.6, -0.075, -0.6]    # from info printed in yt.load

[288 272 288]


In [44]:
# this is from the yt documentation

level = 2
dims = ds.domain_dimensions * ds.refine_by**level

# We construct an object that describes the data region and structure we want
# In this case, we want all data up to the maximum "level" of refinement
# across the entire simulation volume.  Higher levels than this will not
# contribute to our covering grid.
all_data = ds.covering_grid(
    level,
    left_edge=domain_left_edge,
    dims=dims,
    # And any fields to preload (this is optional!)
)

In [50]:
# relevant vars

print(x.units)
print(ds.length_unit)
print(ds.mass_unit)
print(dims)
print(ds.domain_dimensions)
print(ds.refine_by)

code_length
1.0 cm
1.0 g
[1152 1088 1152]
[288 272 288]
2


In [51]:
data_level0 = ds.covering_grid(
    0,
    left_edge=domain_left_edge,
    dims=ds.domain_dimensions
)

In [52]:
# check shape
data_level0['flash', 'dens'].shape

(288, 272, 288)

In [61]:
# we can create a df for any any field in ds.fields.flash
ye = pd.DataFrame(data_level0['flash', 'ye'][:, :, 0])
tion = pd.DataFrame(data_level0['flash', 'tion'][:, :, 0])

In [56]:
density = pd.DataFrame(data_level0['flash', 'dens'][:, :, 0])
trad = pd.DataFrame(data_level0['flash', 'trad'][:, :, 0])

In [58]:
trad.head()

,0,1,2,3,4,5,6,7,8,9,...,262,263,264,265,266,267,268,269,270,271
0,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,...,222811.109375,222809.390625,222807.718750,222806.078125,222804.531250,222803.046875,222801.656250,222800.406250,222799.343750,222798.578125
1,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,...,222812.625000,222810.890625,222809.187500,222807.531250,222805.921875,222804.375000,222802.921875,222801.562500,222800.359375,222799.390625
2,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,...,222814.375000,222812.609375,222810.890625,222809.187500,222807.531250,222805.937500,222804.406250,222802.968750,222801.625000,222800.531250
3,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,...,222816.203125,222814.421875,222812.671875,222810.937500,222809.250000,222807.593750,222806.015625,222804.500000,222803.078125,222801.859375
4,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,21290.070312,...,222818.078125,222816.281250,222814.500000,222812.750000,222811.015625,222809.328125,222807.671875,222806.078125,222804.625000,222803.343750


In [51]:
density.to_csv("dens_grid.csv", index=False)